In [21]:
%load_ext autoreload
%autoreload 2

# check how often let alone and much less occur in datasets


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [22]:
from datasets import load_dataset

ds = load_dataset("ltg/babylm-2024-baby-cosmo-fine-100m", split="train")
# ds = load_dataset("ltg/babylm-2024-baby-cosmo-fine-10m", split="train")
# matches = ds.filter(lambda example: "Biden" in example["article"])
#
# print(matches[0]["article"])



Repo card metadata block was not found. Setting CardData to empty.


In [23]:


from pprint import pp

import re


def find_so_that():
    so_ex = ds.filter(lambda x: " so " in x["text"])
    pp(so_ex)

    all = []
    for i, ex in enumerate(so_ex):
        if i > 1000: break

        # print(ex["text"])
        lines = ex["text"].split("\n")
        for l in lines:
            pattern = r"\bso\s+(\w+)\s+that\b"
            matches = re.findall(pattern, l)
            if matches:
                all.append((l, matches))

    return all


res = find_so_that()
print(len(res))








Dataset({
    features: ['text'],
    num_rows: 77180
})
64


In [24]:
pp(res)

[('Genital HPV is the most common sexually transmitted infection in the United '
  'States. It is so pervasive that it has been called the common cold of the '
  'sexually active world.',
  ['pervasive']),
 ('Aaliyah was so excited that she ran straight home to tell her family. They '
  'were thrilled for her and helped her prepare her application. Aaliyah had '
  'worked hard in school and earned good grades in subjects like civil '
  'engineering, urban planning, and social sciences. She also knew how to use '
  'computers really well, which would help her a lot in this internship.',
  ['excited']),
 ('Now it so happened that the Rev. John Mansfield was not famous for '
  'descriptions, but he did draw a certain picture of Kathleen Desmond which '
  'was not in the least like that young lady, but which abundantly satisfied '
  'her child. Her cheeks grew redder than ever as she listened and she panted '
  'slightly as she snuggled against her beloved uncle.',
  ['happened']),
 ('It s

In [25]:
print(len(ds.filter(lambda x: "much less" in x["text"])))
print(len(ds.filter(lambda x: "let alone" in x["text"])))



765
439


In [26]:
i = 1
x = ds.filter(lambda x: "much less" in x["text"])[i]['text'].split("\n")
print(len(x))
z = [y for y in x if 'much less' in y.lower()]



11


In [27]:

def extract_sub_part(sent: str, tgt='much less'):
    much_less_pos = sent.find(tgt)
    if much_less_pos == -1:
        raise Exception("tgt str not found")

    prev_period = sent.rfind(".", 0, much_less_pos)
    if prev_period == -1:
        prev_period = 0
    else:
        prev_period+=1
    next_period = sent.find(".", much_less_pos)
    if next_period == -1:
        next_period = len(sent)
    else:
        next_period += 1

    extracted = sent[prev_period: next_period]
    extracted = extracted.strip()
    return extracted

extract_sub_part(z[0])


"The drug's side effects proved to be both predictable and manageable and toxicity levels were much less than those associated with other anti-cancer agents, the study found."

In [28]:
from affinity.corr_matrix_new import get_logits_for_masked_sent, compute_surprisal_for_logits
from affinity.tokenization import Sentence
from lib.scoring_fns import probability
from typing import List
def check_prob_on_words(sent: str, words: List[str]):
    s = Sentence(sent)  # Sentence class takes care of tokenization

    # prepare sentence with mask at correct location
    # punctuation is preserved, but will not be part of the mask
    # this will error if your word is not actually present or if your word is multi-tokenized
    # masked_sent keeps track of what was masked
    res = []
    for w in words:
        masked_sent = s.get_input_with_word_masked(w)

        # obtain the logits at the masked position
        logits = get_logits_for_masked_sent(masked_sent, do_print=False)

        # compute probability of the original word
        surp = compute_surprisal_for_logits(
            masked_sent,
            logits,
            probability
            # surprisal     # can also use surprisal
        )
        res.append(surp)
    return res


In [29]:

extract_sub_part("mention of the word 'mathematics'. Mention numbers, let alone", 'let alone')

'Mention numbers, let alone'

In [30]:


def check_target(tgt_str: str):
    examples_with_much_less = ds.filter(lambda x: tgt_str in x["text"])
    # print(len(examples_with_much_less))
    # print(examples_with_much_less[0])
    valid = []
    for i, ex in enumerate(examples_with_much_less):
        if i > 200: break
        # print(i)
        # print(ex)
        ex_lines = ex["text"].split("\n")
        lines_with_tgt_str = [l for l in ex_lines if tgt_str in l]
        if len(lines_with_tgt_str) == 0:
            print('not found')
            print(ex_lines)
        for l in lines_with_tgt_str:
            sent = extract_sub_part(l, tgt_str)
            if sent == "":
                print(f"parsing\n\t{l}\n\t got empty")
            try:
                probs = check_prob_on_words(sent, tgt_str.split(" "))
            except Exception as e:
                print(" exception:", e)
                # pp(lines_with_tgt_str)
                print(l)
                print('------')
                continue
            if all([p > 0.9 for p in probs]):
                valid.append((sent, probs))
    return valid

v = check_target('let alone')
print(len(v))


 exception: multiple instances of let found in But Neandertals have many distinctive features, and there is no reason why these diseases (or any others) would cause many, let alone all, of these features on even one, let alone many, individuals.; indicate occ
Amazingly, a century after scientists knew otherwise, most creationists still believe that Neandertals were merely modern humans, deformed by diseases such as rickets, arthritis or syphilis. Some, but by no means all, Neandertals have been found with signs of health problems such as arthritis. But Neandertals have many distinctive features, and there is no reason why these diseases (or any others) would cause many, let alone all, of these features on even one, let alone many, individuals. Modern knowledge and experience also contradicts the idea that disease is a cause of Neandertal features, because these diseases do not cause modern humans to look like Neandertals.
------


KeyboardInterrupt: 

In [31]:
all_keys = set()
for x in ds:
    all_keys.update(list(x.keys()))
print(all_keys)

{'text'}


In [33]:
ds_baby = load_dataset("ltg/babylm-2024-baby-cosmo-fine-10m", split="train")
print(len(ds_baby.filter(lambda x: "much less" in x["text"])))
print(len(ds_baby.filter(lambda x: "let alone" in x["text"])))



Repo card metadata block was not found. Setting CardData to empty.


81
44


# npn analysis


In [37]:
from rozlib.libs.common.data.utils_jsonl import read_from_jsonl
from proj.cxs_are_revealed.paper.proj_common.npn_dataset_generation.npn_utils import GPTOutput
from proj.cxs_are_revealed.paper.data_config import BabyLMExp6NPN


# note that this will be automagically cached!
ds = load_dataset("ltg/babylm-2024-baby-cosmo-fine-100m", split="train")
def check_npns():
    gpt_outputs: List[GPTOutput] = read_from_jsonl(BabyLMExp6NPN.npn_gpt_outputs, GPTOutput)
    for o in gpt_outputs:
        npn = o.noun + " " + o.prep + " " + o.noun
        ct = len(ds.filter(lambda x: npn in x["text"].lower()))
        print(npn, ct)

check_npns()

Repo card metadata block was not found. Setting CardData to empty.


short to short 1
short after short 0
short by short 0
short upon short 0
sign to sign 0
sign after sign 0
sign by sign 0
sign upon sign 0
tough to tough 0
tough after tough 0
tough by tough 0
tough upon tough 0
army to army 0
army after army 1
army by army 0
army upon army 0
marriage to marriage 0
marriage after marriage 0
marriage by marriage 0
marriage upon marriage 0
break to break 1
break after break 0
break by break 0
break upon break 0
person to person 217
person after person 1
person by person 3
person upon person 0
twist to twist 0
twist after twist 0
twist by twist 0
twist upon twist 0
above to above 0
above after above 0
above by above 0
above upon above 0
smell to smell 0
smell after smell 0
smell by smell 0
smell upon smell 0
delight to delight 0
delight after delight 0
delight by delight 0
delight upon delight 0
friendly to friendly 1
friendly after friendly 0
friendly by friendly 0
friendly upon friendly 0
green to green 1
green after green 0
green by green 0
green upon g